In [3]:
import os

In [1]:
%pwd

'a:\\Machine-Learning-Project-with-MLflow\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'a:\\Machine-Learning-Project-with-MLflow'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:str
    train_data_path:Path
    test_data_path:Path
    model_name:str
    alpha:float 
    l1_ratio:float 
    target_column:str

In [12]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.model_trainer.root_dir])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        param = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=param.alpha,
            l1_ratio=param.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config

In [18]:
from sklearn.linear_model import ElasticNet
import pandas as pd
import os
import joblib
from ml_project import logger

In [19]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):

        train_data =  pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)

        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        model = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=42 
        )

        model.fit(X_train, y_train)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        

In [21]:
try:
    configuration_manager = ConfigurationManager()
    model_trainer_config = configuration_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-07-18 17:01:05,643 : INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-18 17:01:05,644 : INFO: yaml file: params.yaml loaded successfully]


[2025-07-18 17:01:05,647 : INFO: yaml file: schema.yaml loaded successfully]
[2025-07-18 17:01:05,649 : INFO: created directory at: artifacts/model_trainer]
